# Convolution: Introduction and implementation

Table of Contents

* Definition and theories of convolution.
* Applications on various fields.
* Discret convolution equation.
* Implemented routine of `numpy`, `scipy`.
* 2D convolution implementation.

## Terms and definition

### Definition

Continous version

1-dim

$$(f*g)(x) := \frac{1}{\sqrt{2 \pi}} \int_{-\infty}^\infty g(y)f(x-y) dy$$

Multi-dimension convolution: $n$-dim functions $f,g : \mathbb{F}^n \rightarrow \mathbb{F}$

$$(f*g)(x) := \frac{1}{(2 \pi)^{n/2}} \int_{-\infty}^\infty g(\mathbf{r'})f(\mathbf{r}-\mathbf{r'}) d^n r'$$

> Arfken, G. B., Weber, H.-J., &amp; Harris, F. E. (2013). Mathematical methods for physicists: A comprehensive guide. Academic. p. 986

Discrete version

1-dim

$$(f*g)[n] = \sum_{m=-\infty}^{\infty} f[m]g[n-m]$$

Multi-dimension convolution:
$$(f *^{(n)} g)(\vec{v})\\
= \sum_{t_n=-\infty}^{\infty} \sum_{t_{n-1}=-\infty}^{\infty} \cdots \sum_{t_1=-\infty}^{\infty} f(t_1, t_2, \dots, t_n) g(v_1 - t_1, v_2 - t_2, \dots, v_n - t_n)
$$

### Properties

Below properties are hold for continous and discrete cases

1. Communtivity: $f*g = g*f$
2. Associativity: $f*(g *h) = (f * g) *h$
3. Distributivity: $f*(g+h) = f*g + f*h$
4. Associativity with scalar multiplication: $(\alpha f) *g = \alpha (f*g)$
5. Linear with arbitary function $f$ on function space $\mathbf{F}$:
 
   For arbitary function $f$ on function space $\mathbf{F}$ defined on field $\mathbb{F}$, the operator $(f *)$ is linear operator on $\mathbf{F}$,

    1. $\forall g, h \in \mathbf{F}$, $(f*)(g+h) = (f*)(g) + (f*)(h)$
    2. $\forall g, \in \mathbf{F}, \alpha \in \mathbb{F}$, $(f*)(\alpha g) = \alpha(f*)(g)$

### With Fourier Transform

For Fourier transform, $\mathcal{F}: x \rightarrow t$

$$\mathcal{F}[f](t) = \frac{1}{\sqrt{2 \pi}} \int_{-\infty}^{\infty} f(x) e^{i tx} dx$$

#### Transform to product

Let $F = \mathcal{F}[f], G = \mathcal{F}[g]$

$$\mathcal{F}[(f*g)] = F \cdot G$$
and vice verse

$$f \cdot g = \mathcal{F}^{-1}[F * G]$$

These property is very useful for reducing calculation costs in many cases.

#### Parseval Relation

$$\int_{-\infty}^\infty f(-x) g(x) dx = \int_{-\infty}^\infty F(t) G(t) dt$$

## Applications in various Fields

### Mathematics

#### Polynomial multiplication

Product of two polynomial is a prime example of discrete convolution in their coefficients.

$$p_1(x) = \sum_{i=0}^n a_i x^i \in \mathcal{P}_n$$
$$p_2(x) = \sum_{i=0}^m b_i x^i \in \mathcal{P}_m$$

$$p_3 = p_1 \cdot p_2  = \sum_{i =1}^{nm} c_i x^i\in \mathcal{P}_{nm}$$


$$\mathbf{c} = (\mathbf{a} * \mathbf{b})$$

Note that, before applying form to vector convolution, dimension must be exceeded, so some elements of above vectors could be zero.


For larger dimension of polynomial multiplication, FFT can be applied to accelerate calculation. 

#### Solution of integral equation with transformations

Fedholm equation of first kind with a difference kernel $k(x,t)= k(x-t)$ and an unknown function $\phi$ is

$$f(x) = \int_{-\infty}^{\infty} k(x-t) \phi(t) dt$$ 

It is a convolution equation and with product transform on frequncy domain we can express solution $\phi(t)$ as inverse Fourier transform. Note that below equations ignore Fourier coefficients, $a, b$.

$$f(x) = (k * \phi) (x)\\
\mathcal{F}[f] = F(t) =  \mathcal{F}[k * \phi] = K(t) \cdot \Phi(t)
$$


$$\phi(x) = \mathcal{F}^{-1} \left[\frac{F}{K}\right]$$

For Fedholm equation of second kind with a difference kernel $k(x,t)= k(x-t)$ and an unknown function $\phi$ with constan $\lambda$, 

$$\phi(x) = f(x) + \lambda \int_{-\infty}^{\infty} k(x-t) \phi(t) dt$$ 

Solution with same method of above is

$$\phi(x) = \mathcal{F}^{-1} \left[\frac{F}{1- c \lambda K}\right]$$

where $c$ is a constant depending on Fourier coefficients.

However, the above method **assumes that required transformations exist**. If such transforms does not exist or violate Dirichlet condtions, we can not use.

> Arfken, G. B., Weber, H.-J., &amp; Harris, F. E. (2013). Mathematical methods for physicists: A comprehensive guide. Academic. pp. 1054-1055, 1060

### Image filtering

Sharping, Blowing

#### Edge handling

Edge handling in convolution is a handling edge for calculating convolution near edge of the input, because near the edge of the given data there exist kernel parts requiring outside datas of the input. In mathematical definition and signal analysis, it is not mentioned before you treat discrete calculation. Commonly, this technique is significantly treated in image processing field.

For kernel, $K$ and input $A$, where $K\in M_{5 \times 5}, A \in M_{n \times m}, n, m>5$, and $\epsilon_{ij}$ is a required value for calculating convolution outside of the input.

$$\begin{array}{c|c}
 {
    \begin{matrix}
    \epsilon_{11}, \epsilon_{12}, \epsilon_{13}\\ 
    \epsilon_{21}, \epsilon_{22}, \epsilon_{23}\\
    \epsilon_{31}, \epsilon_{32}, \epsilon_{33}
    \end{matrix}
 }&{
   \begin{matrix}
    \epsilon_{14}, \epsilon_{15}\\ 
    \epsilon_{24}, \epsilon_{25}\\
    \epsilon_{34}, \epsilon_{35}
    \end{matrix}
 } \\
 \hline
 {
   \begin{matrix}
    \epsilon_{41}, \epsilon_{42}, \epsilon_{43}\\ 
    \epsilon_{51}, \epsilon_{52}, \epsilon_{53}\\
    \end{matrix}
 }&{
   \begin{matrix}
    a_{11},  a_{12}\\ 
    a_{21},  a_{22}\\
   \end{matrix}
 }
\end{array}$$


<table align = "center" >
<th>Name</th>
<th>Example</th>
<th>Name</th>
<th>Example</th>
<tr>
  <td>Extend</td>
  <td align = "center"> 
  
  $\begin{array}{c|c}
 {
    \begin{matrix}
    a_{11}, a_{11}, a_{11}\\ 
    a_{11}, a_{11}, a_{11}\\
    a_{11}, a_{11}, a_{11}
    \end{matrix}
 }&{
   \begin{matrix}
    a_{11}, a_{12}\\ 
    a_{11}, a_{12}\\
    a_{11}, a_{12}
    \end{matrix}
 } \\
 \hline
 {
   \begin{matrix}
    a_{11}, a_{11}, a_{11}\\ 
    a_{21}, a_{21}, a_{21}\\
    \end{matrix}
 }&{
   \begin{matrix}
    a_{11},  a_{12}\\ 
    a_{21},  a_{22}\\
   \end{matrix}
 }
\end{array}$

  </td>
  <td>Wrap</td>
  <td align = "center">

  $\begin{array}{c|c}
 {
    \begin{matrix}
    a_{(n-2) (m -2)}, a_{(n-2) ( m-1)}, a_{(n-2) (m)}\\ 
    a_{(n-1) (m -2)}, a_{(n-1) ( m-1)}, a_{(n-1) (m)}\\
    a_{(n )(m -2)}, a_{  (n) ( m-1)}, a_{(n)(m)}
    \end{matrix}
 }&{
   \begin{matrix}
    a_{n-2 1}, a_{n-2 2}\\ 
    a_{n-1 1}, a_{n-1 2}\\
    a_{n 1}, a_{n2}
    \end{matrix}
 } \\
 \hline
 {
   \begin{matrix}
    a_{1 m-2}, a_{1 m-1}, a_{1m}\\ 
    a_{2 m-2}, a_{2 m-1}, a_{2m}\\
    \end{matrix}
 }&{
   \begin{matrix}
    a_{11},  a_{12}\\ 
    a_{21},  a_{22}\\
   \end{matrix}
 }
\end{array}$

  </td>
</tr>
<tr>
  <td>Reflect</td>
  <td align = "center">
  
  $\begin{array}{c|c}
 {
    \begin{matrix}
    a_{33}, a_{32}, a_{31}\\ 
    a_{23}, a_{22}, a_{21}\\
    a_{13}, a_{12}, a_{11}
    \end{matrix}
 }&{
   \begin{matrix}
    a_{31}, a_{32}\\ 
    a_{21}, a_{22}\\
    a_{11}, a_{12}
    \end{matrix}
 } \\
 \hline
 {
   \begin{matrix}
    a_{13}, a_{12}, a_{11}\\ 
    a_{23}, a_{22}, a_{12}\\
    \end{matrix}
 }&{
   \begin{matrix}
    a_{11},  a_{12}\\ 
    a_{21},  a_{22}\\
   \end{matrix}
 }
\end{array}$

  </td>
  <td>Mirror</td>
  <td align = "center">
  
  $\begin{array}{c|c}
 {
    \begin{matrix}
    a_{44}, a_{43}, a_{42}\\ 
    a_{34}, a_{33}, a_{32}\\
    a_{24}, a_{23}, a_{22}
    \end{matrix}
 }&{
   \begin{matrix}
    a_{41}, a_{42}\\ 
    a_{31}, a_{32}\\
    a_{21}, a_{22}
    \end{matrix}
 } \\
 \hline
 {
   \begin{matrix}
    a_{14}, a_{13}, a_{12}\\ 
    a_{24}, a_{23}, a_{22}\\
    \end{matrix}
 }&{
   \begin{matrix}
    a_{11},  a_{12}\\ 
    a_{21},  a_{22}\\
   \end{matrix}
 }
\end{array}$

  </td>
</tr>
<tr>
  <td>Constant: For constant </td>
  <td align = "center">
  
  $\begin{array}{c|c}
 {
    \begin{matrix}
    c, c, c\\ 
    c, c, c\\
    c, c, c
    \end{matrix}
 }&{
   \begin{matrix}
    c, c\\ 
    c, c\\
    c, c
    \end{matrix}
 } \\
 \hline
 {
   \begin{matrix}
    c, c, c\\ 
    c, c, c\\
    \end{matrix}
 }&{
   \begin{matrix}
    a_{11},  a_{12}\\ 
    a_{21},  a_{22}\\
   \end{matrix}
 }
\end{array}$

  </td>

  <td>Cropping</td>
  <td> 
    Usually, cropping input data or kernel also discussed in edge handling topic.<br> 
    However, cropping method can be combined with the above methods or the kernel <br>
    cropping can be implemented with constant method (c=0), therefore it will be treated <br>
    in further section as preprocessig option in convolution. <br>
    Kernel cropping is not different with mathematical definition of discrete convolution  </td>
</tr>

</table>


#### Cropping extend

Cropping is defining a minimum overlapping dimension between kernel and input data.
$(c_r, c_c)$ tuple indicates row and column cropping level respectively. In addition, $1 \leq c_r \leq l, 1\leq c_c \leq k$ for dimension of kernel $H \in M_{l \times k}$.
Default convolutions defined on signal and mathematics regard $(c_r, c_c) = (1, 1)$.
Next example shows overlapping of kernel and data which calculate $[1,1]$ element of convolution for $(1,1)$ and $(2, 3)$ cropping condition respectively,

$$\begin{array}{cccc|c}
  c& c& c& c& c\\
  c& c& c& c& c\\
  c& c& c& c& c\\
  c& c& c& c& c\\
  \hline
  c& c& c& c& c*a_{11}
\end{array}, 
\begin{array}{cc|ccc}
 c& c& c& c& c\\
 c& c& c& c& c\\
 c& c& c& c& c\\
 \hline
 c& c& c*a_{11}& c*a_{12}& c*a_{13}\\
 c& c& c*a_{21}& c*a_{22}& c*a_{23}\\
\end{array}$$

The outside of the input data usually omitted in many cases, $0$ element unless some edge handling method is defined.
Considering cropping coefficients $c_r, c_c$, overall dimension of convolution equation is

$$(n, m) * (l, k) = (n+l -2c_r +1, m+k -2 c_c +1) \\
=( (n +2l-2c_r ) -l +1, (m +2k-2c_c ) -k +1)\\
= ( n' -l +1, m' -k +1)$$

That is,

$$X * H, (c_r, c_c) = X' * H, (1,1)$$

$$X' := \left[
\begin{array}{c c c}
{} & R_1 &{}\\
\hline
C_1 &  X & C_2\\
\hline
{}& R_2 & {}
\end{array}\right]$$

where, $\dim(X) = (n ,m), \dim(H) = (l, k), \dim(X') = \left(n +2(l-c_r) , m +2(k-c_c)\right)$. 
The rectangle matrix $R_1 R_2, C_1, C_2$ are symmetric for each row and column filp operation and determined by edge handling method.

For example, see details of next convolution with cropping $(2, 2)$ and kernel cropping: $0$ constant handling.

$$C = \left(\begin{bmatrix}
a & b & c & d\\
e & f & g & h\\
i & j & k & l
\end{bmatrix} * \begin{bmatrix}
1 & 2 & 3\\
4 & 5 & 6\\
7 & 8 & 9
\end{bmatrix}\right) \in M_{(3) \times (3)}$$

The dimension of convolution is $3 = 3+3 -2*2+1, 4 = 4+3 -2*2 +1$ and some elements are weight combination of kernel covered area of input data.

* $C_{1,1} = 5*a + 6*b+ 8*e+ 9*f$
* $C_{1,2} = 4*a + 5*b+ 6*c+ 7*e + 8*f + 9*g$
* $C_{2,1} = 2*a + 3*b+ 5*e+ 6*f + 8*i + 9*j$
* $C_{3,3} = 1*g +2*h + 4*k + 5*l$

And this is same with next convolution with $(c_r, c_c) = (l, k) = (3, 3)$

$$C = \left(\begin{bmatrix}
0& 0 & 0 & 0 & 0& 0\\
0& a & b & c & d& 0\\
0& e & f & g & h& 0\\
0& i & j & k & l& 0\\
0& 0 & 0 & 0 & 0& 0
\end{bmatrix} * \begin{bmatrix}
1 & 2 & 3\\
4 & 5 & 6\\
7 & 8 & 9
\end{bmatrix}\right) \in M_{(3) \times (3)}$$


## Discrete convolution equation

$$\mathbf{A} * \mathbf{X} = \mathbf{X} * \mathbf{A} = \mathbf{B}$$

where, $\mathbf{A} \in M_{n \times m}(\mathbf{F}), \mathbf{X} \in M_{l \times k}(\mathbb{F}), \mathbf{B} \in M_{p \times q}(\mathbb{F})$. Dimension of convolution, $\mathbf{B}$ is differ by the cropping coefficient, $c_r, c_c$. 

$$n-l +1 \leq p= (n +l - 2c_r +1) \leq n+l +1\\
m-k +1 \leq q=(m+k -2c_c +1)  \leq m+k +1$$

Common method is a DFT. Solve the equation on frequncy domain and applying inverse DFT on them.

$$DFT[\mathbf{A}] \cdot DFT[\mathbf{X}] = DFT[\mathbf{B}]\\
\mathbf{X} = IDFT[DFT[\mathbf{B}] / DFT[\mathbf{A}]]$$

Since, convolution is a linear operation we can find corresponding matrix equation of the given convoltuion.
Matrix representation system corresponding to the above convolution is,

$$\mathbf{A_c} \cdot \mathbf{X_c} = \mathbf{B_c}$$

or

$$\mathbf{X'_c} \cdot \mathbf{A'_c} = \mathbf{B'_c}$$

Moreover, the representation with matrix multiplication allows us larger freedom in choice of methods, including optimization methods. Allowing some transformation on $\mathbf{B_c}$, we can choose various matrix equation, matrix-matrix or matrix-vector forms can be presented by the transformations we choose. 
No matter the shape of system, $\mathbf{A_c}$ is always blocked Toeplitz matrix and some special case, it becomes Toeplitz square matrix. However, the transformed matrix equation has much larger dimension then convolution operation. In a view of spatial efficience and calculation time, convolution representaion will be more efficent. 
About $\mathbf{X_c}$ and $\mathbf{B_c}$, they may need reshaping functions for each transform methods. 
For example, Michal and Krystian suggested $\mathbf{B_c} = \mathbf{B}$ transform in 2-dim convolution calculation which calculation result is same with original convolution. 

> Michal Gnacij, Krystian Lapa, Using Toeplitz matrices to obtain 2D convolution, posted: October 27th, 2022, doi:https://doi.org/10.21203/rs.3.rs-2195496/v1


### Convert to matrix vector form:

#### Method 1: $X * K \rightarrow \mathbf{K} \cdot \vec{x}$ 

For the given convolution,

$$K * X = X * K = B$$

and cropping coefficient $(c_r, c_c)$, denote that $K'$ and $X'$ are extended matrix by cropping coefficient as same method in cropping extending.

Then for $X \in M_{n_o \times m_o}$ and $K \in M_{l \times k}$

$$X' \in M_{n \times m}$$
$$K' \in M_{l \times k}$$

$$n = n_o + 2(l - c_r)$$
$$m = n_o + 2(k - c_c)$$

$\mathbf{K} \in M_{(n-l+1)(m-k+1) \times nm} (\mathbb{F}), \vec{x} \in \mathbb{F}^{nm}$


$$X' = \left[\begin{array}{}
\vec{x_1} \\
\hline \vec{x_2} \\
\hline \vdots \\
\hline \vec{x_{n-1}}\\
\hline \vec{x_{n}}
\end{array}\right], \vec{x} = \left[\begin{array}{}
\vec{x_1}^t \\
\hline \vec{x_2}^t \\
\hline \vdots \\
\hline \vec{x_{n-1}}^t\\
\hline \vec{x_{n}}^t
\end{array} \right]$$

$$ \vec{x_i} = [ x_{i1}, x_{i2}, \dots, x_{i (m-1) }, x_{i m}]$$

$$\mathbf{K} = \left[ \begin{array}{c|c|c|c|c|c|c}
K_{11}     & K_{12} & \cdots & K_{1,n_{ext}} & 0 &\cdots&{0} \\
\hline
\mathbf{0} & K_{11} & \cdots & K_{1,n_{ext}-1}& K_{1,n_{ext}} & \cdots&{0}\\
\hline
\vdots & \vdots& \ddots & \ddots & \vdots&{\ddots}&\vdots\\
\hline
\mathbf{0} & \mathbf{0} & \cdots &  K_{11}& K_{12}& \cdots &K_{1,n_{ext}}
\end{array}\right]$$

$$(K_{1i})[p, q] = \begin{cases} 
 k_{i, q-p +1} &  k_{i, q-p +1} \in K' \\
 0 & q-p <0 | q-p\geq k
\end{cases} $$

$K_{1i} \in M_{(m-k+1) \times m}$ thus, $\mathbf{K} \in M_{((n-l+1)(m-k+1))\times(nm)}$

For example, $K_{1i}$ is

$$\begin{bmatrix}
k_{i1} & k_{i2} & k_{i3}&\cdots & k_{ij}   & 0         & \cdots & 0 &\cdots & 0\\
0      & k_{i1} & k_{i2}&\cdots & k_{ij-1} & k_{ij}    & \cdots & 0 &\cdots & 0\\
0      & 0      & k_{i1}&\cdots & k_{ij-2} & k_{ij-1}  & \cdots & 0 &\cdots & 0\\
\vdots & \vdots & \vdots& \ddots& \vdots   & \vdots    & \ddots & {}&\vdots & {}\\
0      & 0      & 0     & \cdots& k_{i1}   & k_{i2}    & \cdots & {k_{ij-1}}& \cdots &0 \\
0      & 0      & 0     & \cdots& 0        & k_{i1}    & \cdots & {k_{ij-2}}& \cdots &0 \\
\vdots & \vdots & \vdots& \ddots& \vdots   & \vdots    & \ddots & {} & {\vdots} & {} \\
0      & 0      & 0     & \cdots& 0        & 0         & \cdots & k_{i1} &\cdots & k_{ij}
\end{bmatrix}$$


#### Condition for square matrix

<table style="border-radius:8px;width:100%;">
<th style="text-align:center;background-color:rgb(0, 0, 0); color:white; border-top-left-radius: 10px;width:20%;">
Thm</th>
<th style="text-align:left;">
Square condition of transformed matrix of convolution </th>
<tr style="text-align:center;">
<td colspan="2">

For given convolution,

$$ X * K$$

where $X \in M_{n\times m},  K \in M_{l \times k}$ with cropping coefficient $(c_r, c_c)$,

corresponding matrix equation with the above tranformation, 

$$\mathbf{K} \cdot \vec{\mathbf{x}}.$$ 

If $l = 2 c_r -1$ and $k = 2 c_c -1$ and cropping, the system is a square system of which $\mathbf{K} \in M_{nm \times nm}$

</td>
</tr>

</table>

**Proof**

Start from extend matrix, $X_{ext}$ by the kernel and cropping dimensions,

Note: It is a roundabout way but very convinence in flow.

$$X' := \left[
\begin{array}{c c c}
{} & R_1 &{}\\
\hline
C_1 &  X & C_2\\
\hline
{}& R_2 & {}
\end{array}\right]$$


$$X'\in M_{n\times m}$$
$$= n_o + 2 e_r, m = m_o + 2 e_c$$

where, $e_r = l- c_r, e_c = k -c_c$.

The corresponding matrix system is 

$$\mathbf{K} \in M_{a\times b}\\
a = (n-l+1)(m-k+1)\\
b = n m$$

First with kernel cropping, the additional $e_r = e_c =0$, thus $n = n_o , m = m_o$.

and if we extend $a$ with $n, m, l, k, c_r, c_c$ then,

$$a = n_o m_o $$
$$ + \left( (n_o+m_o) + (n_o k + m_o l) - lk +1 \right)$$
$$ - \left( 2(n_o c_c + m_o cr) -(l c_c + k c_r) + (c_r + c_c)\right)$$

$$\because l = 2c_r -1, k = 2c_c -1 $$

3-rd therm of RHS becomes

$$- \left((n_o+m_o) + (n_o k + m_o l) - lk +1 \right)$$

$$ \therefore a = n_o m_o, a=b$$

---
 
By the definition of transform, the block banded matrix $\mathbf{K}$ of form,

$$\mathbf{K} = \left[ \begin{array}{c|c|c|c|c|c|c}
K_{11}     & K_{12} & \cdots & K_{1,n_{ext}} & 0 &\cdots&{0} \\
\hline
\mathbf{0} & K_{11} & \cdots & K_{1,n_{ext}-1}& K_{1,n_{ext}} & \cdots&{0}\\
\hline
\vdots & \vdots& \ddots & \ddots & \vdots&{\ddots}&\vdots\\
\hline
\mathbf{0} & \mathbf{0} & \cdots &  K_{11}& K_{12}& \cdots &K_{1,n_{ext}}
\end{array}\right]$$

is reduced to 

$$\mathbf{K} = \left[ 
  \begin{array}{c|c|c|c}
   K_{c_r} & K_{c_r +1} & \cdots & K_{l}\\
   \hline
   K_{c_r -1} & K_{c_r} & \cdots & K_{l-1}\\
   \hline
   \vdots & \vdots & \ddots & \vdots\\
   \hline
   K_1 & K_2 & \cdots  & K_{c_r}
  \end{array}
\right]$$


where, $K_{i} \in M_{m \times m}$, $c_r =m, l = 2m-1$

## Implementations of Discrete convolution

### Basic libraries

* `numpy.convolve` : 1dim
* `scipy`
  * `signal`
    * `convolve` : 1 dim
    * `convolve2d` : 2 dim
  * `ndimage`
    * `convolve` : 2 dim

### Extending

`numpy` 1-dim to 2-dim convolution.

In [1]:
import numpy as np
from typing import Tuple, Literal

In [2]:
def convolve2d(  ## 1dim with 2dim convolution
    data:np.ndarray, 
    filter:np.ndarray,
    crop:Tuple[int, int]=(1, 1),
    edge_mode:Literal["extend", "wrap", "mirror","constant"]="constant",
    edge_params = [0],
    ):
    filter = np.flip(filter,axis=0) # axis=1 flip occured at np.convolve routine.
    l, k = filter.shape
    cr, cc =crop
    er = l - cr
    ec = k - cc

    data_ext = _expand_matrix(data, (er, ec), [edge_mode]+edge_params) # see additonal file for source code
    n,m = data_ext.shape

    #filter = np.flip(filter)
    result = np.zeros(shape = (n-l+1, m-k+1))
    for i, h_row in enumerate(filter):
        data_i = data_ext[i:n-l+i+1]
        result += np.stack([np.convolve(r, h_row, mode="vaild") for r in data_i])
    return result

$$[1, 0, 0, 1, 0, 1, 0, 1] * [1, 0, 0, 1, 0, 1, 0, 1]^t$$
$$= \begin{bmatrix}
1 & 0 & 0 & 1 & 0 & 1 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
1 & 0 & 0 & 1 & 0 & 1 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0  \\
1 & 0 & 0 & 1 & 0 & 1 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0  \\
1 & 0 & 0 & 1 & 0 & 1 & 0 & 1 \\
\end{bmatrix}$$

In [3]:
x1  = np.array([[1, 0, 0, 1, 0, 1, 0, 1]])
convolve2d(x1, x1.transpose())

array([[1., 0., 0., 1., 0., 1., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 1., 0., 1., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 1., 0., 1., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 1., 0., 1., 0., 1.]])

$$K * X$$

In [5]:
K = np.array(
    [[2, 0, 0, 3, 0],
     [2, 4, 4, 4, 0],
     [2, 4,  8, 4, 0],
     [2, 4, -1, 4, 0],
     [2, 0, 0, 0, 0]])
X = np.array(
    [[0, 1, 2],
     [3, 4, 5],
     [6, 7, 8]])

Commute

In [6]:
convolve2d(X, K)

array([[  0.,   2.,   4.,   0.,   3.,   6.,   0.],
       [  6.,  10.,  18.,  21.,  24.,  23.,   0.],
       [ 18.,  36.,  62.,  82.,  77.,  52.,   0.],
       [ 18.,  60., 126., 155., 118.,  60.,   0.],
       [ 18.,  60., 119., 140., 103.,  52.,   0.],
       [ 18.,  46.,  48.,  49.,  20.,  32.,   0.],
       [ 12.,  14.,  16.,   0.,   0.,   0.,   0.]])

In [7]:
convolve2d(K, X)

array([[  0.,   2.,   4.,   0.,   3.,   6.,   0.],
       [  6.,  10.,  18.,  21.,  24.,  23.,   0.],
       [ 18.,  36.,  62.,  82.,  77.,  52.,   0.],
       [ 18.,  60., 126., 155., 118.,  60.,   0.],
       [ 18.,  60., 119., 140., 103.,  52.,   0.],
       [ 18.,  46.,  48.,  49.,  20.,  32.,   0.],
       [ 12.,  14.,  16.,   0.,   0.,   0.,   0.]])

Compare to scipy routine

* `signal.convovle2d`:
* `ndimage.convovle` : It matches the output dimension to input, (input, kernel).

In [4]:
from scipy.signal import convolve2d as conv2d
from scipy.ndimage import convolve as convnd_image

In [10]:
conv2d(X, K) - convolve2d(K, X) # all values must be zero

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

In [14]:
convnd_image(X, K, mode="constant", cval=0)

array([[ 62,  82,  77],
       [126, 155, 118],
       [119, 140, 103]])

In [15]:
K.shape

(5, 5)

In [17]:
convolve2d(X, K, crop=(3, 3))

array([[ 62.,  82.,  77.],
       [126., 155., 118.],
       [119., 140., 103.]])